In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

!pip install -q -U keras-tuner
import kerastuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [4]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [5]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [7]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.001.



In [8]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 7:03 - loss: 2.5678 - accuracy: 0.06 - ETA: 13s - loss: 1.2963 - accuracy: 0.5861 - ETA: 7s - loss: 1.0207 - accuracy: 0.654 - ETA: 5s - loss: 0.9069 - accuracy: 0.68 - ETA: 4s - loss: 0.8518 - accuracy: 0.70 - ETA: 4s - loss: 0.8100 - accuracy: 0.72 - ETA: 3s - loss: 0.7687 - accuracy: 0.73 - ETA: 3s - loss: 0.7493 - accuracy: 0.73 - ETA: 3s - loss: 0.7243 - accuracy: 0.74 - ETA: 2s - loss: 0.7056 - accuracy: 0.75 - ETA: 2s - loss: 0.6908 - accuracy: 0.75 - ETA: 2s - loss: 0.6737 - accuracy: 0.76 - ETA: 2s - loss: 0.6595 - accuracy: 0.77 - ETA: 2s - loss: 0.6494 - accuracy: 0.77 - ETA: 2s - loss: 0.6388 - accuracy: 0.77 - ETA: 2s - loss: 0.6323 - accuracy: 0.78 - ETA: 2s - loss: 0.6253 - accuracy: 0.78 - ETA: 1s - loss: 0.6158 - accuracy: 0.78 - ETA: 1s - loss: 0.6077 - accuracy: 0.78 - ETA: 1s - loss: 0.6051 - accuracy: 0.79 - ETA: 1s - loss: 0.5991 - accuracy: 0.79 - ETA:

60000/60000 [==============================] - ETA: 3s - loss: 0.2339 - accuracy: 0.90 - ETA: 3s - loss: 0.3204 - accuracy: 0.87 - ETA: 3s - loss: 0.3189 - accuracy: 0.88 - ETA: 3s - loss: 0.3219 - accuracy: 0.88 - ETA: 3s - loss: 0.3252 - accuracy: 0.88 - ETA: 3s - loss: 0.3190 - accuracy: 0.88 - ETA: 3s - loss: 0.3187 - accuracy: 0.88 - ETA: 3s - loss: 0.3139 - accuracy: 0.88 - ETA: 3s - loss: 0.3052 - accuracy: 0.88 - ETA: 2s - loss: 0.3082 - accuracy: 0.88 - ETA: 2s - loss: 0.3089 - accuracy: 0.88 - ETA: 2s - loss: 0.3090 - accuracy: 0.88 - ETA: 2s - loss: 0.3079 - accuracy: 0.88 - ETA: 2s - loss: 0.3064 - accuracy: 0.88 - ETA: 2s - loss: 0.3075 - accuracy: 0.88 - ETA: 2s - loss: 0.3077 - accuracy: 0.88 - ETA: 2s - loss: 0.3086 - accuracy: 0.88 - ETA: 2s - loss: 0.3106 - accuracy: 0.88 - ETA: 2s - loss: 0.3108 - accuracy: 0.88 - ETA: 2s - loss: 0.3091 - accuracy: 0.88 - ETA: 2s - loss: 0.3109 - accuracy: 0.88 - ETA: 2s - loss: 0.3116 - accuracy: 0.88 - ETA: 2s - loss: 0.3128 - accu

60000/60000 [==============================] - ETA: 5s - loss: 0.1880 - accuracy: 0.93 - ETA: 2s - loss: 0.2815 - accuracy: 0.89 - ETA: 2s - loss: 0.2509 - accuracy: 0.90 - ETA: 2s - loss: 0.2434 - accuracy: 0.90 - ETA: 2s - loss: 0.2607 - accuracy: 0.90 - ETA: 2s - loss: 0.2542 - accuracy: 0.90 - ETA: 2s - loss: 0.2537 - accuracy: 0.90 - ETA: 2s - loss: 0.2595 - accuracy: 0.90 - ETA: 2s - loss: 0.2624 - accuracy: 0.90 - ETA: 2s - loss: 0.2621 - accuracy: 0.90 - ETA: 2s - loss: 0.2607 - accuracy: 0.90 - ETA: 2s - loss: 0.2610 - accuracy: 0.90 - ETA: 1s - loss: 0.2638 - accuracy: 0.90 - ETA: 1s - loss: 0.2629 - accuracy: 0.90 - ETA: 1s - loss: 0.2636 - accuracy: 0.90 - ETA: 1s - loss: 0.2633 - accuracy: 0.90 - ETA: 1s - loss: 0.2633 - accuracy: 0.90 - ETA: 1s - loss: 0.2640 - accuracy: 0.90 - ETA: 1s - loss: 0.2626 - accuracy: 0.90 - ETA: 1s - loss: 0.2638 - accuracy: 0.90 - ETA: 1s - loss: 0.2670 - accuracy: 0.90 - ETA: 1s - loss: 0.2674 - accuracy: 0.90 - ETA: 1s - loss: 0.2671 - accu

60000/60000 [==============================] - ETA: 5s - loss: 0.1865 - accuracy: 0.90 - ETA: 2s - loss: 0.2294 - accuracy: 0.91 - ETA: 2s - loss: 0.2301 - accuracy: 0.92 - ETA: 2s - loss: 0.2305 - accuracy: 0.91 - ETA: 2s - loss: 0.2370 - accuracy: 0.91 - ETA: 2s - loss: 0.2371 - accuracy: 0.91 - ETA: 2s - loss: 0.2367 - accuracy: 0.91 - ETA: 2s - loss: 0.2339 - accuracy: 0.91 - ETA: 2s - loss: 0.2345 - accuracy: 0.91 - ETA: 2s - loss: 0.2340 - accuracy: 0.91 - ETA: 2s - loss: 0.2335 - accuracy: 0.91 - ETA: 1s - loss: 0.2326 - accuracy: 0.91 - ETA: 1s - loss: 0.2277 - accuracy: 0.91 - ETA: 1s - loss: 0.2276 - accuracy: 0.91 - ETA: 1s - loss: 0.2312 - accuracy: 0.91 - ETA: 1s - loss: 0.2305 - accuracy: 0.91 - ETA: 1s - loss: 0.2303 - accuracy: 0.91 - ETA: 1s - loss: 0.2305 - accuracy: 0.91 - ETA: 1s - loss: 0.2312 - accuracy: 0.91 - ETA: 1s - loss: 0.2333 - accuracy: 0.91 - ETA: 1s - loss: 0.2323 - accuracy: 0.91 - ETA: 1s - loss: 0.2335 - accuracy: 0.91 - ETA: 1s - loss: 0.2337 - accu